In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI模型花园：Google专有模型图像物体检测

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_proprietary_image_object_detection.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>

  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_proprietary_image_object_detection.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/notebooks/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/model_garden/model_garden_proprietary_image_object_detection.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
在Vertex AI Workbench中打开
    </a>
  </td>
</table>

注意：此笔记本已在以下环境中进行了测试：

* Python 版本 = 3.9

## 概述

本笔记本演示如何在[Vertex AI Model Garden](https://cloud.google.com/model-garden) 中使用Google专有的图像对象检测模型进行训练/部署。

### 目标

* 使用Vertex SDK训练新模型

* 测试训练好的模型
  * 在[Vertex AI模型注册表](https://cloud.google.com/vertex-ai/docs/model-registry/introduction)中查看训练好的模型
  * 部署上传的模型
  * 运行预测

* 清理资源

### 成本

此教程使用Google Cloud的可计费组件：

* Vertex AI
* Cloud Storage

了解[Vertex AI定价](https://cloud.google.com/vertex-ai/pricing)和[Cloud Storage定价](https://cloud.google.com/storage/pricing)，并使用[Pricing计算器](https://cloud.google.com/products/calculator/) 根据您的预计使用情况生成成本估算。

### 数据集

本教程使用的数据集是来自[TensorFlow Datasets](https://www.tensorflow.org/datasets/catalog/overview) 的 [OpenImages dataset](https://www.tensorflow.org/datasets/catalog/open_images_v4) 中的 Salads 类别。这个数据集不需要任何特征工程。您在本教程中将使用的数据集版本存储在一个公共Cloud Storage存储桶中。训练好的模型能够预测图像中五种物品类别中沙拉、海鲜、番茄、烘焙食品或奶酪的边界框位置和相应类型。

在开始之前

In [ ]:
! pip3 install --upgrade google-cloud-aiplatform

# Automatically restart kernel after installs
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)
if "google.colab" in str(get_ipython()):
    from google.colab import auth as google_auth

    google_auth.authenticate_user()

### 设置您的Google Cloud项目

**以下步骤是必需的，无论您使用什么笔记本环境。**

1. [选择或创建一个Google Cloud项目](https://console.cloud.google.com/cloud-resource-manager)。当您首次创建账户时，您将获得$300的免费信用用于计算/存储成本。

2. [确保您的项目已启用计费](https://cloud.google.com/billing/docs/how-to/modify-project)。

3. [启用Vertex AI API和Compute Engine API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component)。

4. 如果您在本地运行此笔记本，则需要安装[Cloud SDK](https://cloud.google.com/sdk)。

5. 在下面的单元格中输入您的项目ID。然后运行该单元格，以确保Cloud SDK在本笔记本中的所有命令中使用正确的项目。

**注意**: Jupyter运行以`!`为前缀的行作为shell命令，并且将以`$`为前缀的Python变量插入这些命令。

In [ ]:
import os

from google.cloud import aiplatform

# The project and bucket are for experiments below.
PROJECT_ID = ""  # @param {type:"string"}
BUCKET_URI = ""  # @param {type:"string"}

# You can choose a region from https://cloud.google.com/about/locations.
# Only regions prefixed by "us", "europe", or "asia" are supported.
REGION = "us-central1"  # @param {type:"string"}
REGION_PREFIX = REGION.split("-")[0]
assert REGION_PREFIX in (
    "us",
    "europe",
    "asia",
), f'{REGION} is not supported. It must be prefixed by "us", "europe", or "asia".'

! gcloud config set project $PROJECT_ID

STAGING_BUCKET = os.path.join(BUCKET_URI, "temporal")

aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=STAGING_BUCKET)

### 定义常量

In [ ]:
OBJECTIVE = "iod"

# Dataset constants.
DATASET_PREFIX = "dataset-iod"

# Training constants.
TRAINING_JOB_PREFIX = "train"
# The image object detection salad dataset used to train the model
DATASET_FILE = "gs://cloud-samples-data/vision/salads.csv"

# Evaluation constants.
EVALUATION_METRIC = "AP50"

DEPLOY_JOB_PREFIX = "deploy"

定义常见的库

In [ ]:
import base64
import os
from datetime import datetime

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from google.cloud import aiplatform
from PIL import Image, ImageColor, ImageDraw, ImageFont


def get_job_name_with_datetime(prefix: str):
    return prefix + datetime.now().strftime("_%Y%m%d_%H%M%S")


def load_img(path):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    return Image.fromarray(np.uint8(img)).convert("RGB")


def display_image(image):
    _ = plt.figure(figsize=(20, 15))
    plt.grid(False)
    plt.imshow(image)


def draw_bounding_box_on_image(
    image, ymin, xmin, ymax, xmax, color, font, thickness=4, display_str_list=()
):
    """Adds a bounding box to an image."""
    draw = ImageDraw.Draw(image)
    im_width, im_height = image.size
    (left, right, top, bottom) = (
        xmin * im_width,
        xmax * im_width,
        ymin * im_height,
        ymax * im_height,
    )
    draw.line(
        [(left, top), (left, bottom), (right, bottom), (right, top), (left, top)],
        width=thickness,
        fill=color,
    )

    # If the total height of the display strings added to the top of the bounding
    # box exceeds the top of the image, stack the strings below the bounding box
    # instead of above.
    display_str_heights = [font.getsize(ds)[1] for ds in display_str_list]
    # Each display_str has a top and bottom margin of 0.05x.
    total_display_str_height = (1 + 2 * 0.05) * sum(display_str_heights)

    if top > total_display_str_height:
        text_bottom = top
    else:
        text_bottom = top + total_display_str_height
    # Reverse list and print from bottom to top.
    for display_str in display_str_list[::-1]:
        text_width, text_height = font.getsize(display_str)
        margin = np.ceil(0.05 * text_height)
        draw.rectangle(
            [
                (left, text_bottom - text_height - 2 * margin),
                (left + text_width, text_bottom),
            ],
            fill=color,
        )
        draw.text(
            (left + margin, text_bottom - text_height - margin),
            display_str,
            fill="black",
            font=font,
        )
        text_bottom -= text_height - 2 * margin


def draw_boxes(image, boxes, class_names, scores, max_boxes=40, min_score=0.05):
    """Overlay labeled boxes on an image with formatted scores and label names."""
    colors = list(ImageColor.colormap.values())
    try:
        font = ImageFont.truetype(
            "/usr/share/fonts/truetype/liberation/LiberationSansNarrow-Regular.ttf", 25
        )
    except OSError:
        print("Font not found, using default font.")
        font = ImageFont.load_default()

    for i in range(min(len(boxes), max_boxes)):
        if scores[i] >= min_score:
            ymin, xmin, ymax, xmax = boxes[i]
            display_str = "{}: {}%".format(class_names[i], int(100 * scores[i]))
            color = colors[hash(class_names[i]) % len(colors)]
            draw_bounding_box_on_image(
                image,
                ymin,
                xmin,
                ymax,
                xmax,
                color,
                font,
                display_str_list=[display_str],
            )
    return image

创建数据集

本教程使用存储在公共云存储桶中的沙拉数据集的版本，使用CSV索引文件。

首先快速查看数据。通过计算CSV索引文件中的行数（`wc -l`）来计算示例的数量，然后查看前几行。

In [ ]:
count = ! gsutil cat $DATASET_FILE | wc -l
print("Number of Examples", int(count[0]))

print("First 10 rows")
! gsutil cat $DATASET_FILE | head

接下来，使用 `create` 方法为 `ImageDataset` 类创建 `Dataset` 资源，该方法接受以下参数：

- `display_name`：`Dataset` 资源的可读名称。
- `gcs_source`：一个或多个数据集索引文件列表，用于将数据项导入到 `Dataset` 资源。
- `import_schema_uri`：数据项的数据标记模式。

此操作可能需要几分钟时间。

In [ ]:
dataset = aiplatform.ImageDataset.create(
    display_name=DATASET_PREFIX + "_salads",
    gcs_source=[DATASET_FILE],
    import_schema_uri=aiplatform.schema.dataset.ioformat.image.bounding_box,
)

print(dataset.resource_name)

## 训练新模型

### 创建和运行训练流程

要训练一个AutoML模型，您需要执行两个步骤：
1. 创建一个训练流程。
2. 运行这个流程。

#### 创建训练流程

通过`AutoMLImageTrainingJob`类创建一个AutoML训练流程，具有以下参数：

- `display_name`：`TrainingJob`资源的可读性名称。
- `prediction_type`：要为其训练模型的任务类型。
  - `classification`：图像分类模型。
  - `object_detection`：图像目标检测模型。
- `model_type`：部署模型的类型。对于图像目标检测，我们目前支持以下类型：
  - `SPINENET`：一个模型，可在 Vertex 模型库中进行图像目标检测训练，并具有可定制的超参数。最适合在 Google Cloud 内部使用，不能外部导出。
  - `YOLO`：一个模型，可以在 Vertex 模型库中进行图像目标检测训练，并具有可定制的超参数。最适合在 Google Cloud 内部使用，不能外部导出。
- `checkpoint_name`：可选。该字段针对模型库模型训练保留，基于提供的预训练模型检查点。
- `trainer_config`：可选。通常与模型库模型训练一同使用，用于传递训练器的定制配置。`anchor_size` 不能与 `YOLO` 一起使用。

  包含所有支持参数的示例：
```
  trainer_config = {
    'global_batch_size': '8',
    'learning_rate': '0.001',
    'optimizer_type': 'sgd',
    'optimizer_momentum': '0.9',
    'train_steps': '10000',
    'accelerator_count': '2',
    'anchor_size': '8',
  }
```
  global_batch_size 应该能够被 accelerator_count 整除。
  optimizer_type 支持的值有 'sgd', 'adam', 'adamw', 'lamb', 'rmsprop', 'lars', 'adagrad' 和 'slide'。
  accelerator_count 支持的值有 '2', '4' 和 '8'。
- `metric_spec`：代表优化指标的字典。字典键是指标 ID，该 ID 由您的训练作业报告，可能的值为 ('loss', 'AP50')，字典值是指标的优化目标('minimize' 或 'maximize')。
例如：`metric_spec = {'loss': 'minimize', 'AP50': 'maximize'}`
- `parameter_spec`：代表要优化的参数的字典。字典键是 `metric_id`，作为命令行关键字参数传递给您的训练作业，字典值是指标的参数规范。支持的参数规范可以在 aiplatform.hyperparameter_tuning 中找到。
```
  from google.cloud.aiplatform.aiplatform import hpt as hpt

  parameter_spec = {
    'learning_rate': hpt.DoubleParameterSpec(min=1e-7, max=1, scale='linear'), \
  }
```
- `search_algorithm`：为研究指定的搜索算法。接受以下之一：
  - `None`：如果不指定算法，您的作业将使用默认的 Vertex AI 算法。默认算法应用贝叶斯优化以在参数空间上进行更有效的搜索以达到最佳解决方案。
  - `grid`：在可能空间内进行简单的网格搜索。如果要指定的试验数量大于可能空间中的点数，这个选项特别有用。在这种情况下，如果不指定网格搜索，Vertex AI 默认算法可能会生成重复的建议。要使用网格搜索，所有参数规格必须是 `IntegerParameterSpec`、`CategoricalParameterSpec` 或 `DiscreteParameterSpec` 类型。
  - `random`：在可能空间内进行简单的随机搜索。
- `measurement_selection`：如果服务自动从先前报告的中间测量中选择最终测量，则指示要使用哪个测量。
  接受：`best`、`last` 根据两个考虑选择此选项：
    - A)：您是否期望您的测量值单调提高？如果是这样，选择 `last`。另一方面，如果您的系统可能会**过度训练**，您期望性能一直提高一段时间，然后开始下降，选择 `best`。
    - B)：您的测量值是否明显的嘈杂和/或不可重现的？如果是这样，`best` 会倾向于过于乐观，选择 `last` 可能更好。如果 (A) 和 (B) 中的任何一个或两者都不适用，则选择哪种选择类型都无关紧要。

In [ ]:
from google.cloud.aiplatform import hyperparameter_tuning as hpt

TRAINER_CONFIG = {
    "global_batch_size": "8",
    "learning_rate": "0.001",
    "train_steps": "10000",
    "accelerator_count": "2",
}
METRIC_SPEC_KEY = "AP50"
METRIC_SPEC_VALUE = "maximize"
SEARCH_ALGORITHM = "random"
MEASUREMENT_SELECTION = "best"
MODEL_TYPE = "SPINENET"  # @param {type:"string"} one of the values ["SPINENET", "YOLO"]

PARAMETER_SPEC = {}
if MODEL_TYPE == "YOLO":
    PARAMETER_SPEC = {
        "learning_rate": hpt.DiscreteParameterSpec(
            values=[0.001, 0.1],
            scale="linear",
        ),
        "weight_decay": hpt.DiscreteParameterSpec(
            values=[0.0001, 0.001],
            scale="linear",
        ),
    }
else:
    PARAMETER_SPEC = {
        "learning_rate": hpt.DiscreteParameterSpec(
            values=[0.001, 0.01], scale="linear"
        ),
        "anchor_size": hpt.DiscreteParameterSpec(values=[2, 4], scale="reverse_log"),
    }

job = aiplatform.AutoMLImageTrainingJob(
    display_name=get_job_name_with_datetime(TRAINING_JOB_PREFIX),
    prediction_type="object_detection",
    model_type=MODEL_TYPE,
    base_model=None,
    trainer_config=TRAINER_CONFIG,
    metric_spec={METRIC_SPEC_KEY: METRIC_SPEC_VALUE},
    parameter_spec=PARAMETER_SPEC,
    search_algorithm=SEARCH_ALGORITHM,
    measurement_selection=MEASUREMENT_SELECTION,
)

print(job)

#### 运行训练管道

接下来，通过调用方法 `run`，使用以下参数来运行DAG以启动训练作业：

- `dataset`: 要训练模型的 `Dataset` 资源。
- `model_display_name`: 训练模型的人可读名称。
- `training_fraction_split`: 用于训练的数据集百分比。
- `test_fraction_split`: 用于测试（保留数据）的数据集百分比。
- `validation_fraction_split`: 用于验证的数据集百分比。
- `budget_milli_node_hours`: (可选) 最大培训时间，以毫小时为单位指定（1000 = 小时）。
- `disable_early_stopping`: 如果为 `True`，训练可能会在服务认为无法进一步改善模型目标测量之前完成使用整个预算。

当完成 `run` 方法时，将返回 `Model` 资源。

训练管道的执行将需要最多60分钟。

In [ ]:
model = job.run(
    dataset=dataset,
    model_display_name=get_job_name_with_datetime("salads"),
    training_fraction_split=0.8,
    validation_fraction_split=0.1,
    test_fraction_split=0.1,
    budget_milli_node_hours=20000,
    disable_early_stopping=False,
)

print("Model is: ", model)

## 测试训练好的模型
这一部分展示了如何测试训练好的模型。
1. 从模型注册表部署模型
2. 运行在线预测

In [ ]:
# @title Deploy model from Model Registry
# Model does not support dedicated deployment resources.
# An n1-standard-4 machine with 1 P100 GPU will be used.

deploy_model_name = get_job_name_with_datetime(DEPLOY_JOB_PREFIX + "_" + OBJECTIVE)
print("The deployed job name is: ", deploy_model_name)

endpoint = model.deploy(
    deployed_model_display_name=deploy_model_name,
    traffic_split={"0": 100},
    min_replica_count=1,
    max_replica_count=1,
)

endpoint_id = endpoint.name
print("endpoint id is: ", endpoint_id)

In [ ]:
# @title Run online predictions

# test image file path from a Cloud Storage bucket
test_filepath = ""  # @param {type:"string"}

with tf.io.gfile.GFile(test_filepath, "rb") as f:
    content = f.read()

# The format of each instance should conform to the deployed model's prediction input schema.
instances = [{"content": base64.b64encode(content).decode("utf-8")}]

prediction = endpoint.predict(instances=instances)

img = load_img(test_filepath)
display_image(img)
print(prediction)

# 运行批量预测
现在您的模型资源已经训练完成，您可以通过调用`batch_predict（）`方法来进行批量预测，需要设置以下参数：

* `job_display_name`：批量预测作业的人类可读名称。
* `gcs_source`：来自Cloud Storage桶的jsonl文件路径，包含一个或多个图像的列表。
* `gcs_destination_prefix`：用于存储批量预测结果的Cloud Storage位置。
* `sync`：如果设置为True，则调用将阻塞，等待异步批处理作业完成。

In [ ]:
# A jsonl file path from a Cloud Storage bucket, with all the to-be-predicted images.
gcs_source = ""  # @param {type:"string"}

batch_predict_job = model.batch_predict(
    job_display_name=get_job_name_with_datetime("flowers_bp"),
    gcs_source=gcs_source,
    gcs_destination_prefix=f"gs://{BUCKET_URI}",
    sync=False,
)
print(batch_predict_job)

# Wait for the batch prediction job to finish
batch_predict_job.wait()


# Get the batch prediction results
import json

import tensorflow as tf

bp_iter_outputs = batch_predict_job.iter_outputs()

prediction_results = list()
for blob in bp_iter_outputs:
    if blob.name.split("/")[-1].startswith("prediction"):
        prediction_results.append(blob.name)

tags = list()
for prediction_result in prediction_results:
    gfile_name = f"gs://{bp_iter_outputs.bucket.name}/{prediction_result}"
    with tf.io.gfile.GFile(name=gfile_name, mode="r") as gfile:
        for line in gfile.readlines():
            line = json.loads(line)
            print(line)
            break

清理

In [ ]:
# Delete the dataset.
if "dataset" in globals():
    dataset.delete()

# Undeploy model and delete endpoint.
if "endpoint" in globals():
    endpoint.undeploy_all()
    endpoint.delete(force=True)

# Delete models.
if "model" in globals():
    model.delete()

# Delete the batch predictio job.
if "batch_prediction_job" in globals():
    batch_predict_job.delete()